In [19]:
import pandas as pd

import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

In [12]:
# Authenticate Google Drive
def authenticate_service_account():
    gauth = GoogleAuth()
    gauth.ServiceAuth()  # Authenticate using the service account credentials
    drive = GoogleDrive(gauth)
    return drive


# Save `saved_data.csv` to Google Drive
def save_to_drive(drive, local_file, drive_file_name, folder_id=None):
    query = f"title='{drive_file_name}' and trashed=false"
    if folder_id:
        query += f" and '{folder_id}' in parents"
    file_list = drive.ListFile({'q': query}).GetList()

    if file_list:
        file = file_list[0]
        file.SetContentFile(local_file)
        file.Upload()
        st.success(f"File '{drive_file_name}' updated in Google Drive.")
    else:
        file_metadata = {'title': drive_file_name}
        if folder_id:
            file_metadata['parents'] = [{'id': folder_id}]
        file = drive.CreateFile(file_metadata)
        file.SetContentFile(local_file)
        file.Upload()
        st.success(f"File '{drive_file_name}' uploaded to Google Drive.")


drive = authenticate_service_account()


save_to_drive(drive, saved_data_file, saved_data_file)
st.success("Data saved to Google Drive!")



Index(['AADHAR No', 'AADHAR No.1', 'Len', 'NAME', 'BENEFICIARY ITEM', 'YEAR',
       'Category', 'Selection Status\nYes / No'],
      dtype='object')

In [28]:
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from google.oauth2 import service_account

creds = service_account.Credentials.from_service_account_file(
    'mnpdatabase-4c0143764944.json',
    scopes=['https://www.googleapis.com/auth/drive']
)
print(creds)
drive_service = build('drive', 'v3', credentials=creds)
print(drive_service)
file_metadata = {
    'name': 'articles.csv',
    'parents': ['1ritUg2jlAaf2GsiKWndw93_i9Y6yy0ll']  # ID of the folder where you want to upload
}

print(file_metadata)
file_path = 'articles.csv'

media = MediaFileUpload(file_path, mimetype='text/csv')

file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
file

{'name': 'articles.csv', 'parents': ['1ritUg2jlAaf2GsiKWndw93_i9Y6yy0ll']}


{'id': '1OKNqgWm8OiI8XaPXyMScSx1lhs5W3zth'}

In [ ]:
# # downloading
# file_id = 'file_id_to_download'
# file_path = 'path/to/save/downloaded/file.txt'
#
# request = drive_service.files().get_media(fileId=file_id)
# fh = io.FileIO(file_path, mode='wb')
# downloader = MediaIoBaseDownload(fh, request)
# done = False
#
# while not done:
#     status, done = downloader.next_chunk()